In [1]:
from itertools import chain
import numpy as np
from midiutil import *
import pygame

import subprocess
from collections.abc import Sequence

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
working_pygame = False

track = 0
channel = 0
#pitch = 120
pitch = 60
pitch1 = 60
time = 0
duration = 1
volume = 100


In [3]:
def blank_midifile():
    MyMIDI = MIDIFile(1)
    return MyMIDI

#for i in range(12):
#    MyMIDI.addNote(track,channel,pitch+i,time+i,duration,volume)

In [4]:
if working_pygame:
    # mixer config
    freq = 44100  # audio CD quality
    bitsize = 16   # unsigned 16 bit
    channels = 2  # 1 is mono, 2 is stereo
    buffer = 1024   # number of samples
    pygame.mixer.init(freq, bitsize, channels, buffer)

    # optional volume 0 to 1.0
    pygame.mixer.music.set_volume(0.8)

    def play_music(midi_filename):
        '''Stream music_file in a blocking manner'''
        clock = pygame.time.Clock()
        pygame.mixer.music.load(midi_filename)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            clock.tick(30) # check if playback has finished
else:
    def play_music(midi_filename):
        subprocess.run(['timidity', midi_filename])

In [5]:
intervals = {'1':0,
             'b1':1,
             '2':2,
             'b3':3,
             '3':4,
             '4':5,
             'b5':6,
             '5':7,
             'b6':8,
             '6':9,
             'b7':10,
             '7':11,
             '8':12
            }

In [6]:
class Scale(Sequence):
    def __init__(self, *constituents):
        super(Scale, self).__init__()
        self._list = constituents
        
    def __repr__(self):
        return f"<{self.__class__.__name__} {self._list}>"
    
    def __len__(self):
        return len(self._list)
    
    def __getitem__(self, ii):
        if isinstance(ii, np.int64):
            ii = int(ii)
        if isinstance(ii, int):
            relative_octave = 0
            while ii < 0:
                relative_octave -= 1
                ii += len(self)
            while len(self._list) <= ii:
                relative_octave += 1
                ii -= len(self)
            return self._list[ii] + 12 * relative_octave
        elif isinstance(ii, slice):
            if ii.start is None:
                ii = slice(0, ii.stop, ii.step)
            if ii.stop is None:
                ii = slice(ii.start, len(self), ii.step)
            if ii.step is None:
                ii = slice(ii.start, ii.stop, 1)
            output = []
            while ii.start < ii.stop:
                output.append(self[ii.start])
                ii = slice(ii.start+ii.step, ii.stop, ii.step)
            return output
        else:
            raise TypeError("index must be int or slice")
    

In [7]:
major = [0, 2, 4, 5, 7, 9, 11] + [12 + x for x in [0, 2, 4, 5, 7, 9, 11]]
major_o = Scale(0, 2, 4, 5, 7, 9, 11)
major_o = Scale(0, 2, 4, 5, 7, 9, 11)


print(len(major_o))

[major_o[x] for x in range(-5, 12)]

7


[-8, -7, -5, -3, -1, 0, 2, 4, 5, 7, 9, 11, 12, 14, 16, 17, 19]

In [8]:
major_o

<Scale (0, 2, 4, 5, 7, 9, 11)>

In [9]:
def major_chord(chord_numeral, jazzness=0):
    # take two octaves of scale, starting on chord_numeral
    scale = major_o[chord_numeral-1:chord_numeral-1+7*2]
    # take every other node and return triad + jazzness
    return scale[::2][:3+jazzness]
major_chord(5, 2)

[7, 11, 14, 17, 21]

In [278]:
[major_o[x] for x in range(1, 8)]

[2, 4, 5, 7, 9, 11, 12]

In [279]:
def modal_chord(chord_numeral, jazzness=0, mode = None):
    if mode is None:
        mode = 0
    mode = [major_o[x] for x in range(mode, 2*7+mode)]
    # take two octaves of scale, starting on chord_numeral
    scale = mode[chord_numeral-1:chord_numeral-1+7*2]
    # take every other node and return triad + jazzness
    return scale[::2][:3+jazzness]
major_chord(5, 2)

[7, 11, 14, 17, 21]

In [287]:
modal_chord(2,1)

[2, 5, 9, 12]

In [286]:
modal_chord(1, 1, mode=1)

[2, 5, 9, 12]

In [11]:
class Tone:
    def __init__(self, root, interval, duration=1, volume=100, parent=None, time=None):
        if parent is None and time is None:
            time = 0
        self.parent = parent
        self.time = time    
        self.root = root
        if type(interval) == str:
            self.interval = intervals[interval]
        else:
            self.interval = interval
        self.duration = duration
        self.volume = volume
        
    #def __getattr__(self, name):
    def __getattribute__(self, name):
        if name == 'time' and self.parent is not None:
            return self.parent.time + self.parent.duration
        else:
            return object.__getattribute__(self,name)
            
    
    def add_note(self, midi_file, track, channel):
        if self.parent is not None:
            time = self.parent.time + self.parent.duration
        elif self.time is not None:
            time = self.time
        else:
            raise ValueError("Parent and/or time was messed with - don't do that!")
        
        midi_file.addNote(track, channel, self.root + self.interval, time, self.duration, self.volume)

In [12]:
class Chord:
    def __init__(self, root, constituents, duration=1, volume=100, parent=None, time=None):
        if parent is None and time is None:
            time = 0
        self.parent = parent
        self.time = time    
        self.root = root
        self.constituents = []
        for interval in constituents:
            if type(interval) == str:
                self.constituents.append(intervals[interval])
            else:
                self.constituents.append(interval)
        self.duration = duration
        self.volume = volume

    def __getattribute__(self, name):
        if name == 'time' and self.parent is not None:
            return self.parent.time + self.parent.duration
        else:
            return object.__getattribute__(self,name)
            
    
    def add_note(self, midi_file, track, channel):
        if self.parent is not None:
            time = self.parent.time + self.parent.duration
        elif self.time is not None:
            time = self.time
        else:
            raise ValueError("Parent and/or time was messed with - don't do that!")
        
        for interval in self.constituents:
            midi_file.addNote(track, channel, self.root + interval, time, self.duration, self.volume)

In [13]:
def interval2pitch(ints):
    return list(map(lambda x: intervals[x], ints))

In [14]:
major_scale = np.array([0,2,4,5,7,9,11])
major_blues = np.array(interval2pitch(['1','2','b3','3','5','6']))
minor_blues = np.array(interval2pitch(['1','b3','4','b5','5','b7']))

In [15]:
scale = major_scale
scale_probabilities = np.eye(12)[scale].sum(0)/len(scale)

In [93]:
# Binary tree rhythm generation
remaining_durations = [16]
binary_tree_durations = []
print("Generating durations")
while len(remaining_durations) > 0:
    current_duration = remaining_durations.pop()
    print(current_duration)
    if current_duration > 4:
        probs = [0, 0.5, 0.5]
    elif current_duration > 1:
        probs = [0.5, 0.25, 0.25]
    else:
        probs = [0.9, 0.05, 0.05]
    print(probs)
    flip = np.random.choice([1, 2, 3], p = probs)
    print(flip)
    if flip == 1:
        binary_tree_durations.append(current_duration)
    elif flip == 2:
        remaining_durations.append(current_duration/2)
        remaining_durations.append(current_duration/2)
    elif flip == 3:
        remaining_durations.append(current_duration/3)
        remaining_durations.append(current_duration/3)
        remaining_durations.append(current_duration/3)
    print(len(remaining_durations))

print("Generating_pitches")
binary_tree_pitches = [np.random.choice(range(0, 12), size = 1, p = scale_probabilities)[0]
                       for _ in binary_tree_durations]

print(binary_tree_durations)


midi_file = blank_midifile()
midi_file.addProgramChange(track, channel+1, 0, 8)
random_music = [Tone(pitch, binary_tree_pitches[0], binary_tree_durations[0], time=0)]
print("Genrating Tone objects")
for interval, duration in zip(binary_tree_pitches[1:], binary_tree_durations[1:]):
    print(f"{interval} for {duration}")
    random_music.append(Tone(pitch, interval, duration, parent=random_music[-1]))

print("Writing to midi_file")
for x in random_music:
    x.add_note(midi_file, track, channel)

Generating durations
16
[0, 0.5, 0.5]
3
3
5.333333333333333
[0, 0.5, 0.5]
3
5
1.7777777777777777
[0.5, 0.25, 0.25]
3
7
0.5925925925925926
[0.9, 0.05, 0.05]
1
6
0.5925925925925926
[0.9, 0.05, 0.05]
1
5
0.5925925925925926
[0.9, 0.05, 0.05]
1
4
1.7777777777777777
[0.5, 0.25, 0.25]
2
5
0.8888888888888888
[0.9, 0.05, 0.05]
1
4
0.8888888888888888
[0.9, 0.05, 0.05]
1
3
1.7777777777777777
[0.5, 0.25, 0.25]
1
2
5.333333333333333
[0, 0.5, 0.5]
2
3
2.6666666666666665
[0.5, 0.25, 0.25]
1
2
2.6666666666666665
[0.5, 0.25, 0.25]
2
3
1.3333333333333333
[0.5, 0.25, 0.25]
3
5
0.4444444444444444
[0.9, 0.05, 0.05]
1
4
0.4444444444444444
[0.9, 0.05, 0.05]
1
3
0.4444444444444444
[0.9, 0.05, 0.05]
1
2
1.3333333333333333
[0.5, 0.25, 0.25]
1
1
5.333333333333333
[0, 0.5, 0.5]
3
3
1.7777777777777777
[0.5, 0.25, 0.25]
3
5
0.5925925925925926
[0.9, 0.05, 0.05]
1
4
0.5925925925925926
[0.9, 0.05, 0.05]
1
3
0.5925925925925926
[0.9, 0.05, 0.05]
1
2
1.7777777777777777
[0.5, 0.25, 0.25]
1
1
1.7777777777777777
[0.5, 0.25,

In [ ]:
def bjorklund(m,k,rhythm = None):
    # generates Euclidean rhythms, i.e. rhythms where beats are spread out as evenly as possible across the measure
    # this is basically euclid's algorithm, see http://cgm.cs.mcgill.ca/~godfried/publications/banff.pdf
    if rhythm is None:
        rhythm = k*[[1]] + (m-k)*[[0]]
    #print(rhythm)
    if m%k == 0:
        # flatten list
        return [beat for seq in rhythm for beat in seq]
    else:
        if k>m-k:
            rhythm = [seq+rhythm[-1] if i<(m%k) else seq for i,seq in enumerate(rhythm[:k])]
        else:
            rhythm = [seq+rhythm[-1] if i<k else seq for i,seq in enumerate(rhythm[:k+(m%k)])]
        return bjorklund(k,m%k,rhythm)

In [ ]:
bjorklund(4,1)

In [179]:
def add_chord_progression(chord_progression, beats_per_chord = None, jazz = 0):
    global midi_file
    if beats_per_chord is None:
        beats_per_chord = len(chord_progression)*[4]
    elif type(beats_per_chord) != list:
        beats_per_chord = len(chord_progression)*[beats_per_chord]
        
    chord = Chord(pitch, major_chord(chord_progression[0], jazzness=jazz), duration=beats_per_chord[0])
    chord.add_note(midi_file, track, channel+1)
    for ch,duration in zip(chord_progression[1:],beats_per_chord[1:]):
        chord = Chord(pitch, major_chord(ch, jazzness=jazz), duration=duration, parent=chord)
        chord.add_note(midi_file, track, channel+1)

In [198]:
def add_melody(pitches, rhythm, root = 60):
    random_music = []
    random_music.append(Tone(60, pitches[0], rhythm[0], time=0))
    for interval, duration in zip(pitches[1:], rhythm[1:]):
        random_music.append(Tone(pitch, interval, duration, parent=random_music[-1]))

    for x in random_music:
        x.add_note(midi_file, track, channel)
    return random_music

In [194]:
midi_file = blank_midifile()
midi_file.addProgramChange(track, channel+1, 0, 8)
#add_chord_progression([1,5,4,3,4,1,4,5], beats_per_chord = sum(binary_tree_durations[:4]), jazz=0)
add_chord_progression(()*[6,4,1,5], beats_per_chord = sum(binary_tree_durations[:4]), jazz=0)

In [195]:
add_melody(scale_step_to_pitch(motif_melody), motif_rhythm)

In [119]:
def pitch_to_scale_step(melody):
    scale_step2pitch = {major_o[i]:i for i in range(-20,20)}
    return [scale_step2pitch[pitch] for pitch in melody]

In [120]:
def scale_step_to_pitch(scale_steps):
    return np.array([major_o[scale_step] for scale_step in scale_steps])

In [121]:
def invert(melody):
    melody = pitch_to_scale_step(melody)
    inverted = melody[0] - np.cumsum(np.diff(melody,prepend = melody[0]))
    return scale_step_to_pitch(inverted)

In [122]:
def retrograde(melody):
    melody = pitch_to_scale_step(melody)
    retrograded = np.array(list(reversed(melody)))
    return scale_step_to_pitch(retrograded)

In [123]:
def identity(melody):
    return melody

In [124]:
def repeat_pieces_transformed(melody, transform, piece_length = 4):
    transformed_melody = np.apply_along_axis(transform, 1, melody.reshape(piece_length,-1))
    return np.c_[melody.reshape(piece_length,-1),transformed_melody].flatten()

In [60]:
melody = []
signs = []
#rhythm = []
n_bars = 4

#notes_per_measure, beats_per_measure = 4,16
#notes_per_measure, beats_per_measure = 5,8 # cinquillo
notes_per_measure, beats_per_measure = 5,13 # tresillo
euclidean_rhythm = bjorklund(beats_per_measure, notes_per_measure)
#euclidean_rhythm = bjorklund(13,5) # tresillo
#euclidean_rhythm = bjorklund(2,3)
#euclidean_rhythm = bjorklund(7,3)

# interpret 1's as onsets, and 0's as continuation
rhythm_in_measure = [0.5*(len(i)+1)for i in ''.join(map(str,euclidean_rhythm)).split('1')[1:]]
#rhythm_in_measure = [0.25*(len(i)+1)for i in ''.join(map(str,euclidean_rhythm)).split('1')[1:]]
rhythm = n_bars*rhythm_in_measure
for _ in range(n_bars):
    length = 0
    for i in range(len(rhythm_in_measure)):
    #while length < 4:
        note = np.random.choice(range(0, 12), size = 1, p = scale_probabilities)
        #sign = np.random.choice([-1,1],size=1)
        #sign = 1
        #duration = np.random.choice([0.25,0.5,1,1.25, 1.5, 1.75, 2], size=1)
        #duration = np.random.choice([0.25,0.5,1, 2], size=1)
        
        melody.append(note)
        #signs.append(sign)
        #if duration > (4 - length):
        #    duration = 4 - length
        #rhythm.append(duration)
        #length += duration
melody = np.array(melody).flatten()
rhythm = np.array(rhythm).flatten()
signs = np.array(signs).flatten()

NameError: name 'bjorklund' is not defined

In [ ]:
melody = repeat_pieces_transformed(melody, invert)

In [ ]:
rhythm = repeat_pieces_transformed(rhythm, identity)

In [ ]:
motif = np.array([4,3,4,1])

skeleton = np.array([2,1,2,1,2,1])
#skeleton = np.array([1,2,3,1])
#skeleton = np.array([1,4,2,3,1])

motif_melody = np.tile(motif,len(skeleton))+ skeleton.repeat(len(motif))

motif_melody

len(motif_melody)

motif_melody = repeat_pieces_transformed(scale_step_to_pitch(motif_melody),identity,piece_length=4)

motif_melody

#motif_rhythm = len(skeleton)*binary_tree_durations[:4]
motif_rhythm = (len(motif_melody)//4)*binary_tree_durations[:4]
sum(binary_tree_durations[:4])

In [317]:
def transform_notes(notes, transformations):
    sequence = []
    for n, t in zip(notes, transformations):
        if t == 0:
            sequence.append(0)
            sequence.append(1)
        elif t == 1:
            sequence.append(0)
            sequence.append(0)
        elif t == 2:
            sequence.append(0)
            sequence.append(2)
        elif t == 3:
            sequence.append(1)
            sequence.append(0)
        elif t == 4:
            sequence.append(1)
            sequence.append(1)
        elif t == 9:
            sequence.append(9)
            sequence.append(9)
            
            

    notes_out = []
    durations = []
    current_note = None
    current_duration = None
    
    for ii, event in enumerate(sequence):
        if event == 0:
            if current_note is not None:
                notes_out.append(current_note)
                durations.append(current_duration)
            current_note = notes[ii//2]
            current_duration = 0.5
        elif event == 1:
            current_duration += 0.5
        elif event == 2:
            notes_out.append(current_note)
            durations.append(current_duration)

            current_note = notes[ii//2 + 1]
            current_duration = 0.5
        elif event == 9:
            // Pauser er ikke mulige i vores nuvæ
            pass
    return (notes_out, durations)

In [392]:
gfi_motif = np.array([0,-2,-2,-3])
#gfi_motif = np.array([0,1,2,2])
1//2

0

In [393]:
gfi_skeleton = np.array([1,1,1,1,0,-1])
#gfi_skeleton = np.array([2,2,3,4,2,2])

In [394]:
motif_melody = np.tile(gfi_motif,len(gfi_skeleton))+ gfi_skeleton.repeat(len(gfi_motif))

In [395]:
motif_melody = scale_step_to_pitch(motif_melody)

In [396]:
gfi_transforms = [0, 0, 0, 2, 4, 2, 3, 3,0, 0, 0, 2, 3, 2, 3, 0,0, 2, 3, 2, 3, 3, 3, 0,0]

In [387]:
transforms = np.random.choice(np.arange(5),8*3+1,p=[0.6,0.1,0.1,0.1,0.1])

In [388]:
transformed_melody, transformed_durations = transform_notes(motif_melody, transforms)

In [389]:
midi_file = blank_midifile()
midi_file.addProgramChange(track, channel+1, 0, 8)

In [390]:
add_melody(transformed_melody, transformed_durations, root=53) # 53 is F

In [351]:
gfi_chords = [1,1,2,2,2,5,1]
add_chord_progression(gfi_chords, beats_per_chord = 4, jazz=1)

In [352]:
# common jazz progression II-V-I
#add_chord_progression([2,5,1,1], jazz=1)

# pachabel's canon I-V-vi-iii-IV-I-IV-V
# add_chord_progression([1,5,4,3,4,1,4,5], beats_per_chord = 2, jazz=0)

# four chord song
#add_chord_progression([6,4,1,5], beats_per_chord = 4, jazz=0)
#add_chord_progression([1,5,6,4], beats_per_chord = 4, jazz=0)

# 12-bar blues
#add_chord_progression([1,1,1,1,4,4,1,1,5,4,1,1], beats_per_chord = 2, jazz=0)

# blues
#add_chord_progression([1,4,1,1,4,4,1,1,5,4,1,5], beats_per_chord = 4, jazz=0)

#music = add_melody(melody, rhythm)

In [391]:
for midi_track in midi_file.tracks:
    midi_track.closeTrack()
with open('tmp.mid','wb') as f:
    midi_file.writeFile(f)

play_music('tmp.mid')

In [ ]:
import matplotlib.pylab as plt

In [ ]:
x = np.cumsum(rhythm)

In [ ]:
plt.plot([0] + list(x[:-1]), melody,'.')

In [182]:
midi_file = blank_midifile()
the_lick = []
the_lick.append(Tone(pitch, 1, 0.5, time=0))
for interval, duration in [(2, 0.5), (3, 0.5), (5, 0.5), (2, 1), (-2, 0.5), (0, 1.5)]:
    the_lick.append(Tone(pitch, interval, duration, parent=the_lick[-1]))

for x in the_lick:
    x.add_note(midi_file, track, channel)